# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [17]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [22]:
def average_precision_at_40(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

def average_precision(true_list, predicted_list):
    true_set = frozenset(true_list)
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

def recall(true_list, predicted_list):
    true_set = frozenset(true_list)
    rec = 0
    for i in predicted_list:
      if i in true_set:
        rec += 1
    return rec

In [33]:
import requests
from time import time

url = 'http://35.225.7.222:8080'

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      apa40 = average_precision_at_40(true_wids, pred_wids)
      ap = average_precision(true_wids, pred_wids)
      r = recall(true_wids, pred_wids)
  except:
    pass
  print(f"{q} -> {len(pred_wids)}")
  
  qs_res.append((q, duration, apa40, ap, r))

print(qs_res)
sum_duration = 0
sum_apa40 = 0
sum_ap = 0
sum_r = 0
for tup in qs_res:
  sum_duration += tup[1]
  sum_apa40 += tup[2]
  sum_ap += tup[3]
  sum_r += tup[4]

avg_duration = sum_duration/len(qs_res)
avg_apa40 = sum_apa40/len(qs_res)
avg_ap = sum_ap/len(qs_res)
avg_r = sum_r/len(qs_res)

print(f"average recall is {avg_r}")
print(f"average precision is {avg_ap}")
print(f"average precision at 40 is {avg_apa40}")
print(f"average time is {avg_duration}")

best marvel movie -> 59
How do kids come to world? -> 53
Information retrieval -> 36
LinkedIn -> 46
How to make coffee? -> 60
Ritalin -> 49
How to make wine at home? -> 50
Most expensive city in the world -> 57
India -> 28
how to make money fast? -> 47
Netflix -> 20
Apple computer -> 77
The Simpsons -> 91
World cup -> 63
How to lose weight? -> 42
Java -> 74
Air Jordan -> 48
how to deal with depression? -> 53
How do you make gold -> 48
Marijuana -> 54
How to make hummus -> 45
Winter -> 44
Rick and Morty -> 50
Natural Language processing -> 48
World Cup 2022 -> 26
Dolly the sheep -> 61
Cigarettes -> 82
What is the best place to live in? -> 45
Elon musk -> 32
How do you breed flowers? -> 49
[('best marvel movie', 2.310427188873291, 0.266, 0.266, 3), ('How do kids come to world?', 2.8809807300567627, 0.116, 0.116, 3), ('Information retrieval', 1.2235896587371826, 0.979, 0.979, 12), ('LinkedIn', 0.3314673900604248, 0.877, 0.877, 13), ('How to make coffee?', 1.6079816818237305, 0.598, 0.507,

In [ ]:
try:
  page_rank = requests.post(url + '/get_pagerank', json=[22634890, 23154189, 306742])
  print(page_rank.json())
  page_view = requests.post(url + '/get_pageview', json=[22634890, 23154189, 306742])
  print(page_view.json())
except:
  print("some error")